In [1]:
import cv2
import colorsys
import numpy as np
import os
import sys
import time

from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from keras.utils import multi_gpu_model
from PIL import Image, ImageFont, ImageDraw
from yolo3.model import yolo_body, yolo_eval, tiny_yolo_body
from yolo3.utils import letterbox_image
from yolo import YOLO
from tracker import Tracker
from timeit import default_timer as timer


Using TensorFlow backend.


In [2]:
x1, y1, x2, y2 = -1, -1, -1, -1
path = "../1_02.avi"
font = cv2.FONT_HERSHEY_COMPLEX


In [3]:
def draw_rec(event, x, y, flags, param):
    global x1, y1, x2, y2, drawing, mode
    
    if event == cv2.EVENT_LBUTTONDOWN:
        x1, y1 = x, y
    if event == cv2.EVENT_LBUTTONUP:
        x2, y2 = x, y
        cv2.rectangle(param, (x1, y1), (x2, y2), (0, 0, 255), 3)

In [ ]:
cap = cv2.VideoCapture(path)
# while cap.isOpened():
ret, frame = cap.read()
# cv2.setMouseCallback('draw_rectangle', draw_rec, frame)

yolo = YOLO()
tracker = Tracker(iou_thresh=0.1, max_frames_to_skip=10, max_trace_length=20, trackIdCount=0)
track_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
                    (0, 255, 255), (255, 0, 255), (255, 127, 255),
                    (127, 0, 255), (127, 0, 127)]

count_people = {'people_come_in': 0, 'people_come_out': 0}

# SelectROI (bbox) (top, left, height, width)
bbox_area = cv2.selectROI('track area', frame)
print("Type(bbox_area): {} and value of bbox_area: {}".format(type(bbox_area), bbox_area))
number_frame = 1
start = time.time()
while True:
    ok, frame = cap.read()
    
    image = Image.fromarray(frame)
    
    image, center, box_detected, obj_type = yolo.detect_image(image, bbox_area)
    frame = np.asarray(image)
    
    print("len box_detected: ", len(box_detected))
    if len(box_detected) > 0:
        tracker.Update(box_detected, obj_type)   
        for i in range(len(tracker.tracks)):
            if len(tracker.tracks[i].trace) > 1:
                for j in range(len(tracker.tracks[i].trace) - 1):
                    x1 = tracker.tracks[i].trace[j][0][0]
                    y1 = tracker.tracks[i].trace[j][1][0]
                    x2 = tracker.tracks[i].trace[j + 1][0][0]
                    y2 = tracker.tracks[i].trace[j + 1][1][0]
                    clr = tracker.tracks[i].track_id % 9
                    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), track_colors[clr], 2)
            if len(tracker.tracks[i].trace) >= 9 and (not tracker.tracks[i].counted):
                x_center_first = tracker.tracks[i].trace[0][0][0]
                y_center_first = tracker.tracks[i].trace[0][1][0]
                x_center_second = tracker.tracks[i].trace[7][0][0]
                y_center_second = tracker.tracks[i].trace[7][1][0]
                if y_center_second > y_center_first and x_center_second > x_center_first:
                    count_people["people_come_in"] += 1
                if y_center_second < y_center_first and x_center_second < x_center_first:
                    count_people["people_come_out"] += 1
                    
    x = 30
    y = 30
    dy = 20
    i = 0
    cv2.rectangle(frame, (bbox_area[0], bbox_area[1]), (bbox_area[0] + bbox_area[2], bbox_area[1] + bbox_area[3]), (255, 0, 0), 3)
    FPS = number_frame / float(time.time() - start)
    label_FPS = "FPS: " + str(FPS)
    cv2.putText(frame, label_FPS, (600, x), font, 1, (255, 0, 255), 2, cv2.LINE_AA)
    for key, value in count_people.items():
        text = key + ':' + str(value)
        cv2.putText(frame, text, (x, y + dy * i), font, 1, (255, 0, 255), 2, cv2.LINE_AA)
        i += 1
        
    cv2.imshow("frame", frame)
    number_frame += 1
    k = cv2.waitKey(1) & 0xff
    if k == ord('n'):
        continue
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

../weights/yolo_weights.h5 model, anchors, and classes loaded.
Type(bbox_area): <class 'tuple'> and value of bbox_area: (381, 211, 749, 676)
(416, 416, 3)
Found 3 boxes for img
3.6445925049999914
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09616820800010828
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09120757400000912
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.08929724399990846
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09733243900018351
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09555589199999304
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09603217300013966
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09642780299986953
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09247790900008113
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09759626200002458
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.10397652500000731
len box_detected:  0
(416, 416, 3)

Found 4 boxes for img
person 0.77 (434, 211) (520, 304)
0.09595457500017801
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.76 (443, 211) (514, 306)
0.09389820800015514
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.77 (434, 211) (519, 304)
0.095229713000208
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.80 (443, 211) (514, 305)
0.09248846400009825
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.83 (443, 211) (514, 305)
0.09301444899983835
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.87 (433, 211) (519, 304)
0.09385340900007577
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.85 (443, 211) (515, 306)
0.09308217800003149
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.77 (444, 211) (515, 306)
0.09335682500000075
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.72 (444, 211) (515, 307)
0.0936018590000458
len box_detected:  1
(416, 416, 3)
Foun

Found 6 boxes for img
person 0.58 (711, 211) (788, 278)
person 0.80 (499, 211) (620, 487)
0.09529995999992025
len box_detected:  2
(416, 416, 3)
Found 5 boxes for img
person 0.68 (508, 211) (618, 479)
0.09351427800015699
len box_detected:  1
(416, 416, 3)
Found 5 boxes for img
person 0.78 (521, 211) (619, 484)
0.09188358000005792
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
0.09431400999983452
len box_detected:  0
(416, 416, 3)
Found 5 boxes for img
person 0.42 (552, 215) (683, 496)
0.09097210799995992
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.71 (567, 232) (692, 538)
0.1131130580001809
len box_detected:  1
(416, 416, 3)
Found 5 boxes for img
person 0.43 (756, 212) (842, 272)
person 0.68 (596, 236) (704, 535)
0.09633445999998003
len box_detected:  2
(416, 416, 3)
Found 5 boxes for img
person 0.66 (625, 234) (729, 539)
person 0.74 (773, 211) (860, 264)
0.0940393440000662
len box_detected:  2
(416, 416, 3)
Found 5 boxes for img
person 0.63 (773, 211) (

Found 3 boxes for img
0.1016819779999878
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09905150899999171
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.10099097900001652
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09925864699994236
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.0961800579998453
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09822289300018383
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09832217600001059
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09446998300018095
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09395047800012435
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.10047431900011361
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09768130799989194
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.10127642200018272
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09644117300013022
len box_detected:  0
(416, 416, 3)
F

(416, 416, 3)
Found 4 boxes for img
person 0.87 (535, 219) (672, 459)
0.11310733100003745
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.82 (535, 220) (668, 459)
0.09688686499998767
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.67 (531, 217) (661, 456)
0.09341515500000241
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.79 (537, 211) (648, 444)
0.09377590300005068
len box_detected:  1
(416, 416, 3)
Found 3 boxes for img
0.09513571200000115
len box_detected:  0
(416, 416, 3)
Found 4 boxes for img
person 0.48 (534, 211) (642, 423)
0.09531243500009623
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.51 (521, 211) (618, 418)
0.13313247799987948
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.56 (516, 211) (618, 413)
0.11054307100016558
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.76 (512, 211) (608, 409)
0.0947437339998487
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img


...

Found 4 boxes for img
person 0.89 (478, 211) (581, 327)
0.10758565400010411
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.92 (477, 211) (583, 327)
0.09316979999994146
len box_detected:  1
(416, 416, 3)
Found 5 boxes for img
person 0.43 (474, 212) (534, 303)
person 0.83 (513, 212) (582, 325)
0.10108475599986377
len box_detected:  2
(416, 416, 3)
Found 4 boxes for img
person 0.75 (516, 212) (582, 320)
0.09417140599998675
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.62 (516, 213) (583, 317)
0.0938900730000114
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.86 (523, 211) (593, 318)
0.0938974139999118
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.88 (524, 211) (592, 319)
0.13279513999987103
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.49 (527, 216) (589, 319)
0.09128432199986491
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.51 (479, 219) (586, 316)
0.09134190600002512
len 

(416, 416, 3)
Found 6 boxes for img
person 0.46 (492, 211) (540, 320)
person 0.57 (594, 216) (709, 488)
person 0.97 (381, 211) (450, 330)
0.1100570759999755
len box_detected:  3
(416, 416, 3)
Found 6 boxes for img
person 0.60 (476, 212) (539, 319)
person 0.86 (602, 218) (707, 490)
person 0.97 (381, 211) (449, 330)
0.09779281200007972
len box_detected:  3
(416, 416, 3)
Found 6 boxes for img
person 0.47 (444, 216) (538, 322)
person 0.79 (605, 224) (707, 486)
person 0.96 (381, 211) (449, 325)
0.09549630300011813
len box_detected:  3
(416, 416, 3)
Found 6 boxes for img
person 0.85 (383, 211) (450, 325)
person 0.85 (438, 215) (540, 334)
person 0.97 (607, 212) (721, 503)
0.095237447000045
len box_detected:  3
(416, 416, 3)
Found 6 boxes for img
person 0.52 (383, 211) (453, 326)
person 0.73 (429, 217) (541, 335)
person 0.99 (619, 218) (733, 504)
0.10039584400010426
len box_detected:  3
(416, 416, 3)
Found 5 boxes for img
person 0.68 (424, 217) (538, 333)
person 0.98 (629, 211) (745, 518)
0.10

Found 4 boxes for img
person 0.64 (476, 211) (547, 303)
0.09335046199998942
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.62 (477, 211) (547, 303)
0.09249238000006699
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.54 (477, 211) (547, 303)
0.09235294900008739
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.60 (477, 211) (547, 303)
0.09257331499998145
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.45 (475, 211) (546, 303)
0.09182540200004041
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.41 (475, 211) (545, 304)
0.09510578799995528
len box_detected:  1
(416, 416, 3)
Found 3 boxes for img
0.09483727099996031
len box_detected:  0
(416, 416, 3)
Found 4 boxes for img
person 0.41 (468, 212) (550, 301)
0.09478804700006549
len box_detected:  1
(416, 416, 3)
Found 3 boxes for img
0.14193338500012942
len box_detected:  0
(416, 416, 3)
Found 4 boxes for img
person 0.78 (474, 212) (542, 303)
0.12213010300

(416, 416, 3)
Found 4 boxes for img
person 0.82 (738, 280) (855, 598)
0.09488429700013512
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.98 (750, 286) (865, 614)
0.09775744700004907
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.86 (768, 283) (873, 608)
0.0983938599999874
len box_detected:  1
(416, 416, 3)
Found 3 boxes for img
0.09295081699997354
len box_detected:  0
(416, 416, 3)
Found 4 boxes for img
person 0.52 (785, 300) (890, 644)
0.10059544000000642
len box_detected:  1
(416, 416, 3)
Found 4 boxes for img
person 0.50 (795, 286) (907, 722)
0.1535526459999801
len box_detected:  1
(416, 416, 3)
Found 3 boxes for img
0.1176854140001069
len box_detected:  0
(416, 416, 3)
Found 5 boxes for img
person 0.95 (821, 315) (931, 707)
0.10091776499984917
len box_detected:  1
(416, 416, 3)
Found 5 boxes for img
person 0.75 (832, 318) (943, 737)
0.11994992600011756
len box_detected:  1
(416, 416, 3)
Found 5 boxes for img
person 0.97 (851, 348) (961, 708)
0.

Found 3 boxes for img
0.09735204899993732
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09655307399998492
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.10298401799991552
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09789938099993378
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09781227100006618
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09515172600004007
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09869000399999095
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09743879799998467
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09530709599994225
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.10195860399994672
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.10763659199983522
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.0971795259999908
len box_detected:  0
(416, 416, 3)
Found 3 boxes for img
0.09706360999985009
len box_detected:  0
(416, 416, 3)
